In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy.optimize as spo
import trading_helper as th
import visualize_helper as vh
%load_ext autoreload
%autoreload 2

print('tensorflow version: %s' % tf.__version__)
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)

tensorflow version: 1.12.0
pandas version: 0.23.4
numpy version: 1.15.4


In [2]:
#S&P 100 stocks
markets = ['AAPL','ABBV','ABT','ACN','AEP','AIG','ALL',
    'AMGN','AMZN','APA','APC','AXP','BA','BAC','BAX','BK','BMY','BRKB','C',
    'CAT','CL','CMCSA','COF','COP','COST','CSCO','CVS','CVX','DD','DIS','DOW',
    'DVN','EBAY','EMC','EMR','EXC','F','FB','FCX','FDX','FOXA','GD','GE',
    'GILD','GM','GOOGL','GS','HAL','HD','HON','HPQ','IBM','INTC','JNJ','JPM',
    'KO','LLY','LMT','LOW','MA','MCD','MDLZ','MDT','MET','MMM','MO','MON',
    'MRK','MS','MSFT','NKE','NOV','NSC','ORCL','OXY','PEP','PFE','PG','PM',
    'QCOM','RTN','SBUX','SLB','SO','SPG','T','TGT','TWX','TXN','UNH','UNP',
    'UPS','USB','UTX','V','VZ','WAG','WFC','WMT','XOM']
print(len(markets))
prices = th.get_tickers(markets)

100


In [3]:
cutoffdate = pd.to_datetime('2010-01-01')
prices = prices[prices['date'] > cutoffdate]

In [4]:
prices.head(1)

,close,high,low,open,vol,p,date,ticker
2280,30.5971,30.6429,30.34,30.5071,123432050,0,2010-01-04,AAPL


In [5]:
open_prices, high_prices, low_prices, close_prices, volume = th.get_ohlcv(prices)

In [6]:
daily_close_returns = th.compute_log_returns(close_prices).fillna(0)
prev_returns = th.shift_returns(daily_close_returns, 2).fillna(0)
lookahead_returns = th.shift_returns(daily_close_returns, -1).fillna(0)

In [7]:
stock_ticker = 'FCX'
vh.plot_stock(close_prices[stock_ticker], '{} Stock'.format(stock_ticker))
vh.plot_shifted_returns(
    prev_returns.loc[:, stock_ticker],
    daily_close_returns.loc[:, stock_ticker],
    'Previous Returns of {} Stock'.format(stock_ticker))
vh.plot_shifted_returns(
    lookahead_returns.loc[:, stock_ticker],
    daily_close_returns.loc[:, stock_ticker],
    'Lookahead Returns of {} Stock'.format(stock_ticker))

In [8]:
top_bottom_n = 10
df_long = th.get_top_n(prev_returns, top_bottom_n)
df_short = th.get_top_n(-1*prev_returns, top_bottom_n)
vh.print_top(df_long, 'Longed Stocks', top_bottom_n)
vh.print_top(df_short, 'Shorted Stocks',  top_bottom_n)

10 Most Longed Stocks:
FCX, FB, DVN, HAL, APC, NOV, APA, AMZN, GILD, AAPL
10 Most Shorted Stocks:
FCX, APA, DVN, NOV, HAL, APC, FB, MS, SLB, GILD


In [9]:
expected_portfolio_returns = th.portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)

In [10]:
vh.plot_returns(expected_portfolio_returns.T.sum(), 'Portfolio Returns')

In [11]:
expected_portfolio_returns_by_date = expected_portfolio_returns.T.sum().dropna()
portfolio_ret_mean = expected_portfolio_returns_by_date.mean()
portfolio_ret_ste = expected_portfolio_returns_by_date.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))


Mean:                       -0.000118
Standard Error:             0.000101
Annualized Rate of Return:  -0.14%

